In [6]:
import akshare as ak

bond_cb_redeem_jsl_df = ak.bond_cb_redeem_jsl()

/home/jovyan/data/.venv/lib/python3.11/site-packages/py_mini_racer/py_mini_racer.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [11]:
import backtrader as bt
import pandas as pd
import duckdb


exclude_bonds = tuple(bond_cb_redeem_jsl_df['代码'])

con = duckdb.connect(database=':memory:', read_only=False) 
path = '../dataset/data/convertible_bonds/**/data.parquet'
date = '2025-08-28'

query = f"""
        SELECT 
            *,
            dblow + curr_iss_amt AS rank_indicator
        FROM  read_parquet('{path}')
        WHERE
            NOT bond_nm LIKE '%退%'
            AND NOT price_tips LIKE '%待上市%'
            AND price < 150
            AND premium_rt < 8
            AND rating_cd LIKE '%A%'
            AND qstatus = '00'
            AND update_date = '{date}'
            AND bond_id not in {exclude_bonds}
        ORDER BY rank_indicator ASC
        LIMIT 10
"""

df_all = con.execute(query).df()
con.close()


df_all

,bond_id,bond_nm,bond_py,price,increase_rt,stock_id,stock_nm,stock_py,sprice,sincrease_rt,...,after_next_put_dt,icons,is_min_price,blocked,putting,force_redeem_price_tip,notes,update_date,year_month,rank_indicator
